# LightGlue Demo
In this notebook we match two pairs of images using LightGlue with early stopping and point pruning.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# !unzip "gallery.zip" -d "gallery"

In [3]:
# If we are on colab: this clones the repo and installs the dependencies
from pathlib import Path

if Path.cwd().name != "LightGlue":
    !git clone --quiet https://github.com/cvg/LightGlue/
    %cd LightGlue
    !pip install --progress-bar off --quiet -e .


fatal: destination path 'LightGlue' already exists and is not an empty directory.
/content/drive/MyDrive/LightGlue
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lightglue (pyproject.toml) ... done


In [4]:
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch
import numpy as np
from lightglue import match_pair
from pathlib import Path
import matplotlib.pyplot as plt
import os
import cv2
import sys

/usr/local/lib/python3.10/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/content/drive/MyDrive/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [10]:
torch.set_grad_enabled(False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=300).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint").eval().to(device)
print(device)

cpu


In [12]:
gallery_path = Path("gallery/gallery/gallery")
file_list = os.listdir(str(gallery_path))
feature_path = "gallery/gallery_features/"
feature_list = os.listdir(str(feature_path))
for i, file_name1 in enumerate(file_list):
    percentage = round(i / len(file_list) * 100)
    print("\r文件处理进度: {}%: ".format(percentage), "▓" * (percentage // 2), end="")
    sys.stdout.flush()
    if file_name1 + ".npy" in feature_list:
      print("exites")
      sys.stdout.flush()
      continue
    gallery_image = load_image(gallery_path / file_name1)
    features = extractor.extract(gallery_image.to(device))
    np.save("gallery/gallery_features/" + file_name1 + ".npy", features)
    # features1 = np.load("test/" + file_name1 + ".npy",allow_pickle=True)
    # print(features1)

文件处理进度: 17%:  ▓▓▓▓▓▓▓▓

KeyboardInterrupt: 

In [11]:
feature_path = "gallery/gallery_features/"
file_list = os.listdir(str(feature_path))
print(len(file_list))

['13767.jpg.npy', '13901.jpg.npy', '18266.jpg.npy', '3303.jpg.npy', '6693.jpg.npy', '11162.jpg.npy', '11764.jpg.npy', '27809.jpg.npy', '2208.jpg.npy', '5799.jpg.npy', '28391.jpg.npy', '23152.jpg.npy', '22270.jpg.npy', '17017.jpg.npy', '23677.jpg.npy', '17210.jpg.npy', '17226.jpg.npy', '20271.jpg.npy', '9352.jpg.npy', '10815.jpg.npy', '1779.jpg.npy', '7327.jpg.npy', '14917.jpg.npy', '1530.jpg.npy', '21138.jpg.npy', '21710.jpg.npy', '28325.jpg.npy', '24914.jpg.npy', '5193.jpg.npy', '23354.jpg.npy', '13319.jpg.npy', '20680.jpg.npy', '26577.jpg.npy', '27326.jpg.npy', '9522.jpg.npy', '10385.jpg.npy', '16919.jpg.npy', '3124.jpg.npy', '21610.jpg.npy', '997.jpg.npy', '24952.jpg.npy', '16114.jpg.npy', '16627.jpg.npy', '26907.jpg.npy', '9124.jpg.npy', '25312.jpg.npy', '21101.jpg.npy', '14054.jpg.npy', '19757.jpg.npy', '21144.jpg.npy', '11844.jpg.npy', '7205.jpg.npy', '2464.jpg.npy', '16045.jpg.npy', '2218.jpg.npy', '23855.jpg.npy', '16701.jpg.npy', '23716.jpg.npy', '22487.jpg.npy', '15328.jpg.np

In [ ]:
gallery_path = Path("gallery/gallery/gallery")
file_list = os.listdir(str(gallery_path))
print(file_list)

In [ ]:
torch.set_grad_enabled(False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=1024).eval().to(device)  # load the extractor
matcher = LightGlue(features="superpoint").eval().to(device)
print(device)

Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/superpoint_v1.pth" to /root/.cache/torch/hub/checkpoints/superpoint_v1.pth
100%|██████████| 4.96M/4.96M [00:00<00:00, 331MB/s]
Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/superpoint_lightglue.pth" to /root/.cache/torch/hub/checkpoints/superpoint_lightglue_v0-1_arxiv.pth
100%|██████████| 45.3M/45.3M [00:01<00:00, 40.8MB/s]


cuda


In [ ]:
query_path = Path("gallery/gallery/query_boxed")
gallery_path = Path("gallery/gallery/gallery")

file_list = os.listdir("gallery/gallery/query_boxed/")
file_list.sort(key=lambda x:x.split(".")[0])

for i, file_name in enumerate(file_list):
    query_image = load_image(query_path / file_name)
    # cv2.imread(str(query_path / file_name))
    file_list = os.listdir(str(gallery_path))
    scores = []
    for i, file_name1 in enumerate(file_list):
        percentage = round(i / len(file_list) * 100)
        print("\r文件处理进度: {}%: ".format(percentage), "▓" * (percentage // 2), end="")
        sys.stdout.flush()
        gallery_image = load_image(gallery_path / file_name1)
        _, _, matches = match_pair(extractor, matcher, query_image.to(device), gallery_image.to(device))
        if len(matches['scores']) > 0:
            score = np.mean(np.array(matches['scores']))
        else: score=0
        scores.append((file_name1.split(".")[0], score))
    scores.sort(key=lambda x: x[1], reverse=True)
    with open('ORB_deep.txt', 'a', encoding='utf-8') as file:
        file.write("Q" + str(int(file_name.split('.')[0])+1) + ": ")
        order = ""
        for num, _ in scores:
            order = order + num + " "
        file.write(order)
        file.write("\n")

文件处理进度: 49%:  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓